In [1]:
import numpy as np
from sklearn.datasets import make_spd_matrix
import cvxpy as cp
import time
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
np.random.seed(42)

In [ ]:
def quadform_grad(A, x, mu):  # Complexity of 1st oracle is n + n^2
    return A @ (x - mu) * 2

In [ ]:
def atom_quadform_grad(A, x, mu, i):
    return np.dot(A[:, i], x - mu) * 2

In [ ]:
def constraint_grad(x):
    return 2 * x

In [ ]:
def project_to_constraint(x):
    x_norm = np.linalg.norm(x)
    return x if x_norm <= 1 else x / x_norm

In [ ]:
def is_kkt_optimal(grad_obj, grad_constraint, eps=0.01):

    dot = np.dot(-grad_obj, grad_constraint)
    norm_f = np.linalg.norm(grad_obj)
    norm_g = np.linalg.norm(grad_constraint)
    if norm_f < eps or norm_g < eps:
        return True
    cos_angle = dot / (norm_f * norm_g)

    return bool((np.abs(cos_angle - 1) < eps).all())

In [ ]:
def solve_elipsoid(A, mu):

    x = cp.Variable(A.shape[0])

    objective = cp.Minimize(cp.quad_form(x - mu, A))

    constraints = [cp.norm2(x) <= 1]

    prob = cp.Problem(objective, constraints)

    result = prob.solve()

    return x.value, result

In [ ]:
def str_cvx_gd_solve_elipsoid(
    A, mu, x_init, f_star, eps=0.01, max_iters=10000, return_history=False
):

    alpha = np.min(np.linalg.eigvals(A * 2))

    lr = 2 / alpha

    x = x_init.copy()

    iterations = 0
    operations = 0

    precision_history = [(x - mu).T @ A @ (x - mu) - f_star]

    while iterations < max_iters:

        grad = quadform_grad(A, x, mu)
        x_new = project_to_constraint(x - lr / (iterations + 1) * grad)
        f_val = (x_new - mu).T @ A @ (x_new - mu)

        x = x_new
        iterations += 1

        precision_history.append(abs(f_val - f_star))

        if abs(f_val - f_star) <= eps:
            break

    if return_history:
        return x_new, iterations, f_val, precision_history

    return x_new, f_val, iterations, operations

In [ ]:
def str_cvx_sgd_solve_elipsoid(
    A,
    mu,
    x_init,
    f_star,
    dim_sample_ratio="single", # MAY BE FLOAT in (0, 1]
    eps=0.01,
    max_iters=10000,
    return_history=False,
):

    n_dims = len(mu)

    if dim_sample_ratio == "single":
        m_dims = 1

    else:
        m_dims = int(dim_sample_ratio * n_dims)

    dim_sample_ratio = m_dims/n_dims

    alpha = np.min(np.linalg.eigvals(A * 2))

    lr = 2 / (alpha*dim_sample_ratio)

    x_new = x_init.copy()

    iterations = 0
    operations = 0

    precision_history = [(x - mu).T @ A @ (x - mu) - f_star]

    while iterations < max_iters:

        step_lr = lr / (iterations + 1)

        sampled_dims = np.random.choice(n_dims, size=m_dims, replace=False)

        for i in sampled_dims:
            x_new[i] -= step_lr * atom_quadform_grad(A, x_new, mu, i)

        x_new = project_to_constraint(x_new)

        f_val = (x_new - mu).T @ A @ (x_new - mu)

        x = x_new
        iterations += 1

        precision_history.append(abs(f_val - f_star))

        if abs(f_val - f_star) <= eps:
            break

    if return_history:
        return x_new, iterations, f_val, precision_history

    return x_new, f_val, iterations, operations